<h1 align="center">Práctica 5. Colas de prioridad (montículos binarios o heaps)</h1>
<h3 style="display:block; margin-top:5px;" align="center">Estructuras de datos</h3>
<h3 style="display:block; margin-top:5px;" align="center">Grado en Ciencia de Datos</h3>
<h3 style="display:block; margin-top:5px;" align="center">2023-2024</h3>    
<h3 style="display:block; margin-top:5px;" align="center">Universitat Politècnica de València</h3>
<br>

## Índice
1. ### [Introducción: Objetivos de la práctica](#intro)
1. ### [Actividad 1: Crear la clase `Candidato`](#act1)
1. ### [Actividad 2: Mantener los K mejores candidatos con un `MinHeap`](#act2)
1. ### [Actividad 3: Lectura de datos en formato CSV](#act3)


<a id='intro'></a>
## Introducción

El objetivo de esta práctica es mejorar la comprensión de las colas de prioridad (y, concretamente, las basadas en montículos binarios o *heaps*).

Para ello te planteamos diseñar un algoritmo de tipo [streaming](https://en.wikipedia.org/wiki/Streaming_algorithm) para obtener los K mejores (**mayores**) candidatos a un puesto de trabajo.

Por *streaming* nos referimos a que la lista de los K mejores candidatos se va actualizando al vuelo a medida que se van recibiendo nuevos candidatos de una manera que no hace falta almacenar toda la lista recibida ni esperar a haber recibido toda la lista. En cierto sentido también tiene resemblanza con los algoritmos [anytime](https://en.wikipedia.org/wiki/Anytime_algorithm).

La lista de candidatos utilizada en esta práctica ha sido generada **aleatoriamente** combinando una lista de nombres y otra de apellidos. Los datos están en un fichero en formato [CSV](https://en.wikipedia.org/wiki/Comma-separated_values) del cual mostramos aquí los primeros elementos:

```
26443930I;Jose Borbon Gonzalez;2017-07-09 01:15:55;6.01
48941343S;Pedro Villanueva Sirvent;2016-12-13 21:54:37;8.49
83656115T;Antonio Viñas Ceballos;2019-06-17 21:06:25;7.73
11585957J;Encarnacion Cuesta Palmer;2018-08-07 11:30:13;8.40
92441848E;Antonio Barbero Llort;2016-06-13 00:55:33;7.74
77364312K;Maria Guerra Garcia;2016-06-15 12:02:03;6.87
38872983A;Jose Davila De La Cruz;2015-06-21 22:00:06;5.75
78255561Q;Jose Martinez Peret;2015-09-19 04:38:38;9.32
38427239M;Juan Delpino Ortega;2015-01-20 11:16:08;7.22
68355955K;Christian Diego Ferrera;2015-08-13 12:55:18;7.09
...
```

Como puedes observar, los datos de cada candidato son (en este orden):

- DNI
- Nombre
- Fecha de solicitud del puesto de trabajo
- Nota media de la entrevista de trabajo (la nota era entre 0 y 10, pero ya se han descartado los que tienen nota <5)


El criterio de contratación estipula que, dados dos candidatos, tendrá prioridad:

- El que tenga mayor nota.
- En caso de empate en la nota, el que haya solicitado primero el puesto (fecha de solicitud).

Puedes observar que las solicitudes del fichero CSV **no están ordenadas** (ni por fecha ni por nota).


<a id='act1'></a>
## Actividad 1. Crear la clase `Candidato`

En esta primera actividad debes crear una clase Python llamada `Candidato` que tendrá los datos de cada candidato. 

Aunque en principio podríamos haber utilizado una simple tupla, vamos a crear una clase con estos atributos:

- DNI: una cadena.
- Nombre: otra cadena.
- Fecha: un objeto de la clase [`datetime`](https://docs.python.org/3/library/datetime.html)
- Nota: un valor de tipo `float`.

Además de un constructor que reciba todos esos argumentos, debes sobrecargar (implementar) los siguientes métodos:

- El método `__lt__` asociado al operador `<`. Un candidato será `<` que otro si tiene peor nota o, cuando tenga igual nota, si presentó después su solicitud (tienen prioridad los de mejor nota y luego los que solicitaron antes el puesto de trabajo).
- El método `__repr__` para mostrar un candidato como cadena de texto (puedes mostrarlo en formato CSV para que coincida aprox. con la línea de la cual se leyeron sus datos).

**Ayuda:** 

- para convertir una cadena tipo `'2016-08-18 23:23:11'` a objeto de tipo [`datetime`](https://docs.python.org/3/library/datetime.html) se puede utilizar el siguiente método:

```python
from datetime import datetime

datetime.strptime('2016-08-18 23:23:11', '%Y-%m-%d %H:%M:%S')
```

- Obviamente, para converir una cadena como `'8.74'` a valor numérico (`float`) se puede hacer `float('8.74')`.

In [2]:

from datetime import datetime

class Candidato:
    def __init__(self, dni, nombre, fecha, nota):
        self.dni = dni
        self.nombre = nombre
        self.fecha = datetime.strptime(fecha, '%Y-%m-%d %H:%M:%S')
        self.nota = float(nota)

    def __lt__(self, other):
        if self.nota < other.nota:
            return True
        elif self.nota == other.nota:
            if self.fecha < other.fecha:
                return True
            return False
        return False
    
    def __repr__(self):
        return f'{self.dni};{self.nombre};{self.fecha};{self.nota}'



<a id='act2'></a>
## Actividad 2. Mantener los K mejores candidatos con un `MinHeap`

A continuación vamos a utilizar un `MinHeap` para mantener en todo momento los (hasta un máximo de) `K` mejores candidatos. Para ello utilizaremos el `MinHeap` para localizar **al peor candidato** de la lista y poder sustituirlo por uno nuevo en caso de que el nuevo resulte ser mejor.

Es decir, la idea es tener una lista de candidatos que crecerá hasta la longitud `K`. Una vez tengamos el nº de candiatos `K`, cada vez que llegue un nuevo candidato veremos si reemplaza o no al peor de todos los `K` candidato seleccionados "de momento".

Os proporcionamos un clase `MinHeap` que tiene un atributo adicional para obtener una copia del vector de elementos:

In [3]:
class MinHeap:
    def __init__(self, initial=()):
        self._data = [v for v in initial]
        self._buildheap()

    def __len__(self):
        return len(self._data)

    def get_data(self): # método adicional para evitar tener que acceder a un atributo privado
        return self._data.copy() # crea una copia nueva

    def _heapify(self, pos):
        d = self._data # para simplificar la escritura
        size = len(d)  # para simplificar la escritura
        child_pos = 2*pos + 1 # posición del hijo izquierdo
        if child_pos < size: # existe hijo izquierdo
            # determinar el menor de los hijos:
            if child_pos+1 < size and d[child_pos+1] < d[child_pos]:
                # usaremos la posición del hijo derecho
                child_pos += 1
            # child_pos tiene la posición del menor de los hijos
            if d[pos] > d[child_pos]:
                # intercambiamos con el hijo
                d[pos],d[child_pos] = d[child_pos],d[pos]
                self._heapify(child_pos)

    def _buildheap(self):
        # se trata de ir aplicando _heapify a cada elemento desde el
        # último que tenga hijos hacia atrás hasta llegar a la raíz:
        if len(self._data) > 1: # no hace falta en otro caso
            ultimo_indice = len(self._data)-1
            padre_ultimo = (ultimo_indice-1)//2
            for pos in range(padre_ultimo, -1, -1): # hacia atras hasta 0
                self._heapify(pos)

    def min(self):
        if len(self._data) == 0:
            raise KeyError('MinHeap is empty.')
        return self._data[0]
    
    def remove_min(self):
        if len(self._data) == 0:
            raise KeyError('MinHeap is empty.')
        themin = self._data[0] # el que vamos a devolver
        last = self._data.pop() # quitamos el último
        if len(self._data) > 0: # si había > 1
            self._data[0] = last # sustituimos el 1º por el último
            self._heapify(0) # restaurar propiedad de heap
        return themin # devolvemos el mínimo

    def add(self, value):
        d = self._data # para simplificar la escritura:
        d.append(value) # añadimos el nuevo elemento
        pos = len(d)-1 # índice del nuevo elemento
        parent_pos = (pos-1)//2 # índice de su padre
        while (pos > 0 and d[pos] < d[parent_pos]):
            # los intercambiamos:
            d[pos], d[parent_pos] = d[parent_pos], d[pos]
            # y subimos
            pos = parent_pos
            parent_pos = (pos-1)//2 # índice de su padre


Se pide crear una clase nueva llamada `Kbest` que tenga como atributos:

- El valor `K`
- Un objeto de la clase `MinHeap`

> **Atención:** Observa que la nueva clase `Kbest` no hereda de `MinHeap` sino que tiene una instancia de esta clase como uno de sus atributos.


La clase `Kbest` debe disponer de los siguientes métodos:

- El constructor recibe únicamente el valor `K` y crea un `MinHeap` inicialmente vacío.
- El método `add` permite añadir un nuevo elemento. Debe tener en cuenta que nunca podemos dejar más de `K` candidatos en la lista.
- El método `getKbest` devuelve una lista con (hasta) los K mejores elementos previamente insertados con `add`. Esa lista de (hasta) K mejores elementos no está necesariamente ordenada, pero sí contiene los K mejores elementos.

Decimos "hasta `K`" porque si hemos insertado menos de `K` elementos el tamaño será inferior a `K`.

In [4]:
class Kbest:
    def __init__(self, k):
        self.k = k
        self.heap = MinHeap()
    
    def add(self, candidato):
        if len(self.heap) < self.k:
            self.heap.add(candidato)
        else:
            if self.heap.min() < candidato:
                min = self.heap.remove_min()
                self.heap.add(candidato)
    
    def getKbest(self):
        return self.heap.get_data()

<a id='act3'></a>
## Actividad 3. Lectura de datos en formato CSV

En este último apartado vamos a leer el fichero de candidatos en formato CSV procesándolos de uno en uno y metiéndolos en un objeto `kmen` de la clase `Kbest` (clase creada en la actividad anterior).

Tras procesar todo el fichero, mostraremos la lista de los candidatos ordenada de mayor (mejor) a menor (peor). Esta última etapa de ordenar, como puedes observar más abajo, se realiza aplicando la ordenación estándar de Python al resultado de llamar al método `getKbest`. 

En este apartado debes utilizar el módulo Python [`csv`](https://docs.python.org/3/library/csv.html?highlight=csv#module-csv). Utiliza los ejemplos de la documentación para completar el siguiente código:

In [5]:
import csv

kmen = Kbest(10)
with open('candidatos.csv', newline='') as candidatos:
    reader = csv.reader(candidatos, delimiter=';')
    for row in reader:
        c = Candidato(*row)
        kmen.add(c)

seleccionados = sorted(kmen.getKbest(), reverse=True)

for c in seleccionados:
    print(c)

39501550C;Maria Allo Mendez;2019-04-20 08:14:03;9.79
58938822C;Concepcion Gumbao Iglesias;2018-01-06 17:41:30;9.76
73702065Y;Maria Castillo Lorenzo;2018-11-25 00:50:34;9.73
37981074D;Francisco Contreras Paya;2017-08-14 05:20:39;9.71
63839342O;Manuel Dominguez Garcia;2016-04-16 11:12:08;9.71
68614143P;Cristina Moran Nova;2015-05-26 20:57:42;9.67
76421705M;Miguel Gutierrez Ortiz;2019-05-23 07:31:33;9.63
37762090A;Manuela Tirado Galan;2016-04-11 06:55:48;9.63
88331167H;Marianne Marin Garcia;2015-10-20 22:14:59;9.61
16444692I;Concepcion Camarena Lorenzo;2019-10-17 13:02:26;9.59


> **Nota:** Sale en formato CSV si la implementación del método `__repr__` de la clase `Candidato` así lo hace.

Si el código anterior es correcto, el resultado de mostrar los 10 mejores candidatos ordenados de mayor a menor es:

```python
seleccionados = sorted(kmen.getKbest(),reverse=True)
for c in seleccionados:
    print(c)
```

```
39501550C;Maria Allo Mendez;2019-04-20 08:14:03;9.79
58938822C;Concepcion Gumbao Iglesias;2018-01-06 17:41:30;9.76
73702065Y;Maria Castillo Lorenzo;2018-11-25 00:50:34;9.73
63839342O;Manuel Dominguez Garcia;2016-04-16 11:12:08;9.71
37981074D;Francisco Contreras Paya;2017-08-14 05:20:39;9.71
68614143P;Cristina Moran Nova;2015-05-26 20:57:42;9.67
37762090A;Manuela Tirado Galan;2016-04-11 06:55:48;9.63
76421705M;Miguel Gutierrez Ortiz;2019-05-23 07:31:33;9.63
88331167H;Marianne Marin Garcia;2015-10-20 22:14:59;9.61
16444692I;Concepcion Camarena Lorenzo;2019-10-17 13:02:26;9.59
```